In [3]:
pip install zod

Note: you may need to restart the kernel to use updated packages.


In [4]:
%load_ext autoreload
%autoreload 2
import sys

sys.path.append("../")

# imports for plotting
from matplotlib import pyplot as plt

%matplotlib inline
plt.rcParams["figure.figsize"] = [20, 10]

# import the ZOD DevKit
from zod import ZodFrames
from zod import ZodSequences

# import default constants
import zod.constants as constants
from zod.constants import Camera, Lidar, Anonymization, AnnotationProject

# import useful data classes
from zod.data_classes import LidarData

# NOTE! Set the path to dataset and choose a version
dataset_root = "/mnt/ml-data-storage/jens/zod"
version = "full"  # "mini" or "full"

# initialize ZodFrames
zod_frames = ZodFrames(dataset_root=dataset_root, version=version)

/home/hannalukas/miniconda3/envs/prob/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading infos: 100000it [00:16, 6105.51it/s]


# Train Test Split

In [5]:
# get default training and validation splits 90/10 
training_frames = zod_frames.get_split(constants.TRAIN)
validation_frames = zod_frames.get_split(constants.VAL)

# print the number of training and validation frames
print(f"Number of training frames: {len(training_frames)}")
print(f"Number of validation frames: {len(validation_frames)}")

# print out the first 5 training frames
print("The 5 first training frames have the ids:", sorted(list(training_frames))[:5])


Number of training frames: 89972
Number of validation frames: 10023
The 5 first training frames have the ids: ['000000', '000002', '000003', '000004', '000005']


In [6]:
#Saving test frames to txt file 
file_path = '/home/hannalukas/PROB/PROB_exp/data/OWOD/ImageSets/ZOD/zod_all_task_test.txt'

# Writing the frame IDs to the file
with open(file_path, 'w') as file:
    for frame_id in validation_frames:
        file.write(f"{frame_id}\n")

# Class frequencies

In [ ]:
dynamic_classes = ["Car", "Van", "Truck", "Animal", "Trailer", "Bus", "HeavyEquip", "Wheelchair", "TramTrain", "Bicycle", "Motorcycle", "Stroller", "PersonalTransporter", "Pedestrian"]

# Initialize a dictionary to count the frequency of each class
class_frequency = {class_name: 0 for class_name in dynamic_classes}

for frame in zod_frames: 
    print(frame.metadata.frame_id)
    annotations = frame.get_annotation(AnnotationProject.OBJECT_DETECTION)
    class_names = [annotation.object_type for annotation in annotations]

    #Update the counts 
    for class_name in class_names:
        if class_name in class_frequency:
            class_frequency[class_name] += 1

In [35]:
sorted(class_frequency.items(), key=lambda x: x[1], reverse=True)

[('Car', 1208474),
 ('Pedestrian', 359177),
 ('Bicycle', 124122),
 ('Van', 115042),
 ('Truck', 78795),
 ('Motorcycle', 32197),
 ('PersonalTransporter', 28490),
 ('Bus', 25802),
 ('TramTrain', 13576),
 ('Trailer', 4887),
 ('Stroller', 4199),
 ('HeavyEquip', 4021),
 ('Animal', 3439),
 ('Wheelchair', 467)]

# Splitting the data

* No overlap of images between tasks 
* No instances of subsequent classes in the task

In [46]:
T1_classes = [
    "Car", "Pedestrian", "Bicycle", "Van"
    ]

T2_classes = [
    "Truck", "Trailer"
]

T3_classes = [
    "TramTrain", "Motorcycle", "Stroller", "HeavyEquip"
]

T4_classes = [
    "Bus", "PersonalTransporter", "Animal", "Wheelchair"
]

t1_train = []
t2_train = []
t3_train = []
t4_train = []

Appending to t1_train
* Append images from the training set 
* Append images that dont conatin classes from T2 T3 or T4 
* Append images that contain at least one object from T1

In [ ]:
training_frames = list(training_frames) 

excluded_classes = T2_classes + T3_classes + T4_classes

# For the zod frames that are in the training frames, only append images to t1_train if it does not contain classes T2, T3, T4 but contains at least one class from T1
for frame in zod_frames: 
    print(frame.metadata.frame_id)
    if frame.metadata.frame_id in training_frames:
        annotations = frame.get_annotation(AnnotationProject.OBJECT_DETECTION)
        class_names = [annotation.object_type for annotation in annotations]

        # Check for inclusion of T1_classes and exclusion of excluded_classes
        if any(item in T1_classes for item in class_names) and not any(item in excluded_classes for item in class_names):
            t1_train.append(frame.metadata.frame_id)


In [62]:
print("Number of training frames in T1: ", len(t1_train), "\nTotal number of training frames: ", len(training_frames))

Number of training frames in T1:  30413 
Total number of training frames:  89972


In [54]:
# Save training frames to txt file

file_path = '/home/hannalukas/PROB/PROB_exp/data/OWOD/ImageSets/ZOD/zod_t1_train.txt'

# Writing the frame IDs to the file
with open(file_path, 'w') as file:
    for frame_id in t1_train:
        file.write(f"{frame_id}\n")

Appending to t2_train
* Exlude images from t1_train
* Append images that dont conatin classes from T3 or T4
* Append images that conatin at least one object from T2 